In [1]:
import pandas as pd
import numpy as np
import random
import math

In [12]:
df=pd.read_csv(r'C:\Users\JDD\Downloads\query-hive-475454.csv')
df

,item_a,item_a_title,item_b,item_b_title,score
0,1054327,DL250进藏动力足够。,2780478,豪爵铃木DL250初体验（2）,0.002613
1,1054327,DL250进藏动力足够。,2689001,“老头乐”超适合在山间奔驰 | “不冒险”的冒险旅行车款,0.002539
2,1054327,DL250进藏动力足够。,2284482,全新铃木DL1050 八月国内上市 价格有竞争力,0.002530
3,1054327,DL250进藏动力足够。,3017005,看见重庆摩友发了个，提醒下贵阳摩友别踩坑,0.002518
4,1054327,DL250进藏动力足够。,2928528,『哈罗试车』高金GK500-好玩的大巧克力,0.002512
...,...,...,...,...,...
545695,554550,第一次上山,2644136,三轮摩托车D照考试重点,0.002475
545696,554550,第一次上山,1938263,最强的250双缸量产跑车，38.7马力水冷，极速200km/h,0.002474
545697,554550,第一次上山,3195341,AW500R 又一台颜值超高的仿赛，国产新品牌 杭州『AW』,0.002474
545698,554550,第一次上山,1893989,摩托有必要买那么好的吗？,0.002474


In [13]:
title_dict=dict()
sim_dict=dict()
for i in df.index:
    item_a=df['item_a'][i]
    item_b=df['item_b'][i]
    title_a=df['item_a_title'][i]
    title_b=df['item_b_title'][i]
    score=df['score'][i]
    sim_dict.setdefault(item_a,{})
    sim_dict[item_a][item_b]=score
    title_dict[item_a]=title_a
    title_dict[item_b]=title_b

In [4]:
data=pd.read_csv(r'C:\Users\JDD\Downloads\query-hive-475243.csv')
train=dict()
for i in data.index:
    deviceid=data['deviceid'][i]
    itemid=data['itemid'][i]
    action_type=data['action_type'][i]
    train.setdefault(deviceid,[])
    if itemid not in train[deviceid]:
        train[deviceid].append(itemid)

In [5]:
data=pd.read_csv(r'C:\Users\JDD\Downloads\query-hive-475233.csv')
test=dict()
for i in data.index:
    deviceid=data['deviceid'][i]
    itemid=data['itemid'][i]
    action_type=data['action_type'][i]
    test.setdefault(deviceid,{'expose':[],'click':[]})
    if action_type==0:
        if itemid not in test[deviceid]['expose']:
            test[deviceid]['expose'].append(itemid)
    else:
        if itemid not in test[deviceid]['click']:
            test[deviceid]['click'].append(itemid)

In [6]:
device_list=[x for x in train.keys() if x in test.keys()]
sample_device=random.sample(device_list,50000)
len(device_list)

111464

In [17]:
ACC=[]
RECALL=[]
expose_ACC=[]
for device in sample_device:
    rec_dict={}
    scaned=train[device]
    for item in scaned:
        if item in sim_dict.keys():
            sim_items={k:sim_dict[item][k] for k in list(sim_dict[item].keys())[:50]}
            for k,v in sim_items.items():
                if k in rec_dict.keys():
                    tmp_v=rec_dict[k]
                    rec_dict[k]=max(tmp_v,v)
                else:
                    rec_dict[k]=v
    rec_sorted=sorted(rec_dict.items(),key=lambda x:x[1], reverse=True)[:50]
    rec=[x[0] for x in rec_sorted]
    exposed=[x for x in rec if x in test[device]['expose']]
    if len(exposed)>0 and len(test[device]['click'])>0:
        true=[x for x in rec if x in test[device]['click']]
        po=[x for x in test[device]['click'] if x in rec]
        acc=len(true)/len(rec)
        recall=len(po)/len(test[device]['click'])
        ACC.append(acc)
        RECALL.append(recall)
        expose_acc=len(true)/len(exposed)
        expose_ACC.append(expose_acc)
print(np.mean(ACC),np.mean(RECALL),np.mean(expose_ACC),2*np.mean(RECALL)*np.mean(expose_ACC)/(np.mean(RECALL)+np.mean(expose_ACC)))

0.010980485483103285 0.15094333139838348 0.22716202076876124 0.18137057296596018


In [90]:
expose_rate=[]
for device in sample_device:
    rec_dict={}
    scaned=train[device]
    for item in scaned:
        if item in sim_dict.keys():
            sim_items={k:sim_dict[item][k] for k in list(sim_dict[item].keys())[:50]}
            for k,v in sim_items.items():
                if k in rec_dict.keys():
                    tmp_v=rec_dict[k]
                    rec_dict[k]=max(tmp_v,v)
                else:
                    rec_dict[k]=v
    rec_sorted=sorted(rec_dict.items(),key=lambda x:x[1], reverse=True)[:50]
    rec=[x[0] for x in rec_sorted]
    exposed=[x for x in rec if x in test[device]['expose']]
    if len(exposed)>0 and len(test[device]['expose']):
        rate=len(exposed)/len(test[device]['expose'])
        expose_rate.append(rate)
np.mean(expose_rate)

NameError: name 'mp' is not defined

In [8]:
expose_rate=[]
for device in device_list:
    click_list=test[device]['click']
    if len(click_list)>0:
        common=[x for x in click_list if x in sim_dict.keys()]
        rate=len(common)/len(click_list)
        expose_rate.append(rate)
np.mean(expose_rate)

0.9834008622351011